In [43]:
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer

#### Loading the data set

In [44]:
df = pd.read_csv('../../../Downloads/Amazon Sale Report(in).csv',parse_dates=['Date'])

/var/folders/j6/2w5njgp15332sfj8ybwvwwp40000gn/T/ipykernel_50712/377302366.py:1: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../../../Downloads/Amazon Sale Report(in).csv',parse_dates=['Date'])


In [45]:
df = df.drop(columns=["Unnamed: 22"], errors="ignore")

In [46]:
df.head()

,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,...,Qty,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by
0,0,405-8078784-5731545,2022-04-30,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,...,0,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,NaN,False,Easy Ship
1,1,171-9198151-1101146,2022-04-30,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,...,1,INR,406.00,BENGALURU,KARNATAKA,560085.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship
2,2,404-0687676-7273146,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,...,1,INR,329.00,NAVI MUMBAI,MAHARASHTRA,410210.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True,NaN
3,3,403-9615377-8133951,2022-04-30,Cancelled,Merchant,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,...,0,INR,753.33,PUDUCHERRY,PUDUCHERRY,605008.0,IN,NaN,False,Easy Ship
4,4,407-1069790-7240320,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,...,1,INR,574.00,CHENNAI,TAMIL NADU,600073.0,IN,NaN,False,NaN


In [47]:
# checking for null values

df.isnull().sum()

index                     0
Order ID                  0
Date                      0
Status                    0
Fulfilment                0
Sales Channel             0
ship-service-level        0
Style                     0
SKU                       0
Category                  0
Size                      0
ASIN                      0
Courier Status         6872
Qty                       0
currency               7795
Amount                 7795
ship-city                33
ship-state               33
ship-postal-code         33
ship-country             33
promotion-ids         49153
B2B                       0
fulfilled-by          89698
dtype: int64

In [48]:
df["month"] = df["Date"].dt.month
df["revenue"] = df["Amount"]
df["profit"] = df["revenue"].fillna(0)*0.30

### Feature engineering


In [49]:
numeric_cols = ["Qty","Amount","revenue","profit","month","ship-postal-code"]
categorical_cols = [
    "Status","Fulfilment","ship-service-level",
    "Style","SKU","Category","Size","ASIN",
    "Courier Status","currency","ship-city","ship-state","ship-country","promotion-ids",
    "B2B","fulfilled-by"
]

In [58]:
num_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
])


In [59]:
cat_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
])


In [60]:
preprocessor = ColumnTransformer([
    ("num", num_pipe, numeric_cols),
    ("cat", cat_pipe, categorical_cols)
])


In [61]:
processed = preprocessor.fit_transform(df)

processed_cols = numeric_cols + categorical_cols

clean_df = pd.DataFrame(processed, columns=processed_cols)

# Restore date separately (not transformed)
clean_df["Date"] = df["Date"].values



### Analyticcs on the clean data set 

In [62]:
total_sales = clean_df["revenue"].astype(float).sum()
avg_sales = clean_df["revenue"].astype(float).mean()

sales_by_region = clean_df.groupby("ship-state")["revenue"].sum()

sales_per_customer = clean_df.groupby("ship-postal-code")["revenue"].sum()

top_products = clean_df.groupby("SKU")["revenue"].sum().sort_values(ascending=False).head()

monthly_sales = clean_df.groupby(clean_df["Date"].dt.month)["revenue"].sum()

sales_growth = monthly_sales.pct_change().dropna()*100

total_profit = clean_df["profit"].astype(float).sum()

/var/folders/j6/2w5njgp15332sfj8ybwvwwp40000gn/T/ipykernel_50712/948364606.py:12: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  sales_growth = monthly_sales.pct_change().dropna()*100


In [63]:
# ================= CUSTOMER SEGMENTATION =================

cust_df = sales_per_customer.reset_index()
cust_df.columns = ["customer","sales"]

segment_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("kmeans", KMeans(n_clusters=3, random_state=42))
])

cust_df["segment"] = segment_pipe.fit_predict(cust_df[["sales"]])

In [65]:
# ================= FORECAST =================

forecast_df = monthly_sales.reset_index()
forecast_df["idx"] = np.arange(len(forecast_df))

X = forecast_df[["idx"]]
y = forecast_df["revenue"]

forecast_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("lr", LinearRegression())
])

forecast_pipe.fit(X, y)

next_month = forecast_pipe.predict([[len(X)]])[0]

/Users/sushilpandey/Desktop/python-learning/pyLibraries/lib/python3.13/site-packages/sklearn/utils/validation.py:2691: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [66]:
print("\nTOTAL SALES:", round(total_sales,2))
print("AVERAGE SALES:", round(avg_sales,2))
print("\nSALES BY STATE:\n", sales_by_region)
print("\nTOP PRODUCTS:\n", top_products)
print("\nTOTAL PROFIT:", round(total_profit,2))
print("\nMONTHLY SALES:\n", monthly_sales)
print("\nSALES GROWTH (%):\n", sales_growth)
print("\nNEXT MONTH FORECAST:", round(next_month,2))
print("\nCUSTOMER SEGMENTS SAMPLE:")
print(cust_df.head())



TOTAL SALES: 83308653.3
AVERAGE SALES: 645.93

SALES BY STATE:
 ship-state
ANDAMAN & NICOBAR      167193.62
ANDHRA PRADESH        3446706.72
APO                        605.0
AR                         493.0
ARUNACHAL PRADESH        99470.0
                         ...    
goa                       2467.0
orissa                     597.0
punjab                    9227.0
rajasthan                 3946.0
rajsthan                   521.0
Name: revenue, Length: 69, dtype: object

TOP PRODUCTS:
 SKU
JNE3797-KR-L    555436.77
J0230-SKD-M      548874.2
J0230-SKD-S     494457.14
JNE3797-KR-M    476070.16
JNE3797-KR-S    427872.57
Name: revenue, dtype: object

TOTAL PROFIT: 23577803.49

MONTHLY SALES:
 Date
3      107128.85
4    30653103.32
5    27741396.75
6    24807024.38
Name: revenue, dtype: object

SALES GROWTH (%):
 Date
4    28513.303811
5       -9.498897
6      -10.577594
Name: revenue, dtype: float64

NEXT MONTH FORECAST: 38624158.33

CUSTOMER SEGMENTS SAMPLE:
   customer     sales  se